In [ ]:
import math
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats("svg")
import numpy as np

%matplotlib inline

Hands-on with the ring test model
=================================

This notebook is meant to provide the building blocks for exploring the performance impacts of various NEURON and CoreNEURON options, using the ring test model (`ringtest.py`).

This model uses a custom MOD file (`halfgap.mod`), so we must start by building `special` using it:

In [ ]:
!nrnivmodl mod

Now we can run the `ringtest.py` script, passing any options we want to:

In [ ]:
!nrniv -python ringtest.py -nt 1

The above command executed in around 0.2s on the author's machine.
**Is that good?** *The author has no idea...*

Typically when examining the performance of a new model, or an existing model on a new system, or indeed a change in software version, we need to look at trends and comparisons.

To illustrate this, we will run the same model using different numbers of CPU threads.

First, define a small helper function that runs the `ringtest.py` script with some options and returns the results for use/plotting.
Try not to worry too much about the details here:

In [ ]:
def ringtest(*args, mpi=None, repeat=3):
    """TODO: update ringtest.py to write these somewhere and avoid regexing"""
    import re
    from subprocess import CalledProcessError, check_output, STDOUT

    def run():
        cmd = []
        if mpi is not None:
            cmd += [
                "mpiexec",
                "-bind-to",
                "core:overload-allowed",
                "--oversubscribe",
                "-n",
                str(mpi),
            ]
        cmd.append("nrniv")
        if mpi is not None:
            cmd.append("-mpi")
        cmd += ["-python", "ringtest.py"]
        cmd += [str(x) for x in args]
        try:
            out = check_output(
                cmd,
                shell=False,
                stderr=STDOUT,
                text=True,
            )
        except CalledProcessError as e:
            print(e.stdout)
            raise
        data = {}
        m = re.search("runtime=([0-9\.]+)", out)
        assert m
        data["runtime"] = float(m.group(1))
        m = re.search("load_balance=([0-9\.]+)%", out)
        assert m
        data["load_balance"] = float(m.group(1)) / 100
        return data

    # run the measurements `repeat` times, to get a basic uncertainty estimate
    data = [run() for _ in range(repeat)]
    return {k: np.array([d[k] for d in data]) for k in data[0].keys()}

Now we can record the runtime data using different numbers of threads.
This is steered by the `-nt` option to `ringtest.py`.

In [ ]:
def pows_of_2(max):
    """Given a power of 2 (e.g. 16) return all powers of 2 from 1 to there.

    e.g. pows_of_2(16) -> [1, 2, 4, 8, 16]."""
    return [2**n for n in range(int(math.log2(max)) + 1)]


# Save performance data for several different thread counts from 1 to 16
thread_data = {
    "data": {nt: ringtest("-nt", nt) for nt in pows_of_2(max=32)},
    "label": "Thread parallelism",
}

Now we have gathered the simulation runtimes for different thread values, we can plot these:

In [ ]:
def scaling_plot(data_dicts, ideal_count=8, fname=None):
    """
    Given a list of dicts containing scaling data, plot them on a common scale.
    Also include an idealised scaling curve for `ideal_count` processors.
    If the `fname` argument is not None, save the plot to that file.
    """
    plt.figure()
    plt.xscale("log", base=2)
    plt.xlabel("Thread / MPI rank count")
    plt.ylabel("Simulation runtime [s]")
    all_x, all_y0 = set(), set()
    for data_dict in data_dicts:
        # e.g. one data_dict for multi-threaded measurements
        xvals = sorted(data_dict["data"].keys())
        yvals, yerrs_low, yerrs_high = [], [], []
        for nt in xvals:
            runtime_measurements = data_dict["data"][nt]["runtime"]
            yvals.append(runtime_measurements.mean())
            yerrs_low.append(max(0, yvals[-1] - runtime_measurements.min()))
            yerrs_high.append(max(0, runtime_measurements.max() - yvals[-1]))
            all_x.add(nt)
        all_y0.add(yvals[0])
        plt.errorbar(
            xvals, yvals, yerr=(yerrs_low, yerrs_high), label=data_dict["label"]
        )
    # Also draw an idealised perfect-scaling curve for a machine with `ideal_count` cores
    xvals = sorted(all_x)
    plt.plot(
        xvals,
        min(all_y0) / np.minimum(xvals, ideal_count),
        label="Ideal {} cores".format(ideal_count),
    )
    plt.legend()
    if fname is not None:
        plt.savefig(fname)
    return plt.show()


scaling_plot([thread_data])

Based on this, **how many CPU cores do you think this machine has available?**

> *Aside: this plot is showing the **strong scaling** behaviour of the model: the problem size is remaining constant while the number of parallel processors is varying.*

The `ringtest.py` script supports a lot of other options:

In [ ]:
!nrniv -python ringtest.py --help

As well as thread-based parallelism, we can also use process-based parallelism via MPI.
In this case, we need to run `ringtest.py` using `mpiexec`.

The `ringtest(...)` helper function defined above can handle this via the `mpi=X` keyword argument.

In [ ]:
mpi_data = {
    "data": {num_ranks: ringtest(mpi=num_ranks) for num_ranks in pows_of_2(max=16)},
    "label": "MPI parallelism",
}

In [ ]:
scaling_plot([thread_data, mpi_data])

In [ ]:
def SVGZ(name):
    """Helper function for loading and displaying compressed SVG files."""
    from gzip import GzipFile
    from IPython.display import SVG

    return SVG(data=GzipFile(name + ".svgz").read())

An example version of the above figure is shown below:

In [ ]:
SVGZ("thread_mpi_ref")

Some questions to consider:

* Does your own version of it, generated above, look similar?
  * If not, can you explain the differences?

Other ideas to explore
----------------------

There are a lot of other options to `ringtest.py` that can be used to explore how different [Core]NEURON features affect performance.

For example:
* The `-coreneuron` option enables CoreNEURON execution.
  * With CoreNEURON, `-permute 0|1|2` controls the data ordering used by CoreNEURON.
* The `-multisplit` option enables splitting a single cell across multiple threads (**not** ranks).
* The `-gap` option enables the use of gap junctions (**TODO** add a link to documentation assuming what this means).
* The `-nring`, `-ncell`, `-npt`, `-branch` and `-compart` options control the size and characteristics of the model.
* The `-tstop` option controls how long the simulation runs for.

### Load balancing

The `ringtest.py` script also shows the "load balance" when it is run.
This is defined as the mean computation time across MPI ranks divided by the maximum computation time across ranks.

In a well-balanced model this is 100%, because all MPI ranks spend all their time computing (not waiting for the other ranks) and the mean and max are the same.
Conversely, if there are $N$ ranks and one of them is much slower than the other $N-1$, the load balance would be $\frac{1}{N}$ (*i.e.* for $N=4$ the load balance is a number between $25\%$ and $100\%$).

This is an example of a mismatched model:

In [ ]:
data = ringtest("-nring", 1, "-ncell", 1, "-compart", 16000, 16000, "-tstop", 1, mpi=4)
data["load_balance"].mean()

On the author's machine this prints `0.25`, so with $N=4$ we are getting the worst possible load balance.

**Can you see why?**

### Single cell simulation

In this example we are running `ringtest.py` with a single cell (`nring=1` and `ncell=1`) that is quite complex (`branch=5000` and `compart=5`):

In [ ]:
single_cell_args = (
    "-nring",
    1,
    "-ncell",
    1,
    "-npt",
    1,
    "-branch",
    5000,
    5000,
    "-compart",
    5,
    5,
    "-tstop",
    5,
)
single_cell_thread_data = {
    "data": {nt: ringtest("-nt", nt, *single_cell_args) for nt in pows_of_2(max=8)},
    "label": "Thread parallelism",
}

In [ ]:
scaling_plot([single_cell_thread_data])

Does this model benefit from using multiple threads? **Do you know why?**

**What NEURON feature could we use to take advantage of multiple threads in this model?**

*Next part is a spoiler/solution*

In [ ]:
single_cell_multisplit_thread_data = {
    "data": {
        nt: ringtest("-nt", nt, "-multisplit", *single_cell_args)
        for nt in pows_of_2(max=8)
    },
    "label": "Multisplit + threads",
}

In [ ]:
scaling_plot([single_cell_thread_data, single_cell_multisplit_thread_data])

### Other topics

Some other questions that might be interesting:
* Is CoreNEURON (`-coreneuron`) faster than NEURON?
  * When is the difference smaller / larger?
  * If you have an NVIDIA GPU on your machine we could try the `-gpu` option (but this is a little more involved...)